In [1]:
import math

import pandas as pd

from lol_fandom import get_tournaments
from lol_fandom import get_scoreboard_games


pd.set_option('display.max_columns', None)

In [2]:
class Team:
    """Team information"""
    q = math.log(10) / 400

    def __init__(self, name, league):
        self.name = name
        self.league = league
        self.win = 0
        self.loss = 0
        self.r = 1000
        self.RD = 350
        self.last_game_date = None

    def update_team_name(self, name):
        self.name = name

    def update_league(self, league):
        self.league = league

    def update_last_game_date(self, game_date):
        self.last_game_date = game_date

    @classmethod
    def get_g(cls, RDi):
        """Compute g(RDi)

        Args:
            RDi (float): Ratings Deviation (RD)

        Returns:
            float: g(RDi)
        """
        return 1 / math.sqrt(1 + (3 * cls.q ** 2 * RDi ** 2) / math.pi ** 2)

    @classmethod
    def get_e(cls, r0, ri, g):
        """Compute E(s|r0, ri, RDi)

        Args:
            r0 (float): previous rating
            ri (float): rating of opponent
            g (float): g(RDi)

        Returns:
            float: E(s | r0, ri, RDi)
        """
        return 1 / (1 + 10 ** ((g * (r0 - ri)) / -400))

    @classmethod
    def get_d(cls, g, e):
        """Compute d^2

        Args:
            g (float): g(RDi)
            e (float): E(s | r0, ri, RDi)

        Returns:
            float: d^2
        """
        return 1 / (cls.q ** 2 * g ** 2 * e * (1 - e))

    def init_rd(self):
        """Initialize RD"""
        self.RD = 350

    @classmethod
    def update_point(cls, team1, team2, result):
        """Update ratings of team1 and team2

        Args:
            team1 (Team): Team1
            team2 (Team): Team2
            result (int): 1 is Team1 win, 0 is Team2 win
        """
        assert isinstance(team1, Team)
        assert isinstance(team2, Team)

        team1_r = team1.r
        team2_r = team2.r
        team1_RD = team1.RD
        team2_RD = team2.RD

        team1._update_point(team2_r, team2_RD, result)
        team2._update_point(team1_r, team1_RD, 1 - result)

    def _update_point(self, ri, RDi, s):
        if s == 1:
            self.win += 1
        else:
            self.loss += 1

        g_RD = Team.get_g(RDi)
        e = Team.get_e(self.r, ri, g_RD)
        d_2 = Team.get_d(g_RD, e)
        self.r = self.r + Team.q / (1 / self.RD ** 2 + 1 / d_2) * g_RD * (s - e)

        self.RD = math.sqrt((1 / self.RD ** 2 + 1 / d_2) ** -1)

    def get_win_prob(self, opponent):
        """Get win probability

        Args:
            opponent (Team): Opponent team

        Returns:
            float: Win probability
        """
        return Team.get_e(self.r, opponent.r, Team.get_g(opponent.RD))

    def to_dict(self):
        """Make Team class to dictionary

        Returns:
            dict: Dictionary of Team instance
        """
        data = {
            'Team': self.name,
            'League': self.league,
            'Win': self.win,
            'Loss': self.loss,
            'WinRate': self.win / (self.win + self.loss) if self.win != 0 else 0,
            'r': self.r,
            'RD': self.RD,
            'last_game_date': self.last_game_date,
        }

        return data

def get_team_id(teams_id, name):
    return teams_id.loc[teams_id['team'] == name, 'team_id'].iloc[0]

def proceed_rating(teams_id, teams, games):
    """Proceed rating with teams and games

    Args:
        teams_id (DataFrame): ID of teams 
        teams (list): List of Team instances
        games (DataFrame): Scoreboard games
    """
    for row in games.itertuples():
        team1, team2 = row.Team1, row.Team2
        game_date = row._6
        result = 1 if row.WinTeam == team1 else 0
        id1, id2 = get_team_id(teams_id, team1), get_team_id(teams_id, team2)
        Team.update_point(teams[id1], teams[id2], result)
        teams[id1].update_last_game_date(game_date)
        teams[id2].update_last_game_date(game_date)

def get_rating(teams):
    """Get rating of teams

    Args:
        teams (list): List of Team instances

    Returns:
        DataFrame: Rating of teams
    """
    ratings = pd.DataFrame(data=map(lambda x: x.to_dict(), teams.values()))
    ratings = ratings.sort_values(by='r', ascending=False).reset_index(drop=True)
    return ratings

def get_team_name(same_team_names, name):
    """Get latest name of the team

    Args:
        same_team_names (dict): Dictionary of names of same teams
        name (str): Name of the team

    Returns:
        str: Latest name of the team
    """
    while name in same_team_names:
        name = same_team_names[name]
    return name

def is_proper_league(league):
    if 'WCS' == league or 'MSI' == league:
        return False
    return True

In [3]:
teams_id = pd.read_csv('./csv/teams_id.csv')
teams_id

,team,team_id
0,100 Thieves,253
1,17 Academy,586
2,1UP Gaming,202
3,2144 Danmu Gaming,91
4,269 Gaming,506
...,...,...
694,n!faculty,168
695,paiN Gaming,262
696,team propertybb2,139
697,vVv Gaming,213


In [4]:
teams = {}
for year in range(2011, 2024):
    tournaments = pd.read_csv(f'./csv/tournaments/{year}_tournaments.csv')
    scoreboard_games = pd.read_csv(f'./csv/scoreboard_games/{year}_scoreboard_games.csv')
    for page in tournaments['OverviewPage']:
        print(f'{page} rating ...')
        sg = scoreboard_games.loc[scoreboard_games['OverviewPage'] == page]
        print(f'\t{sg.shape[0]} matches')

        league = sg['League'].iloc[0]
        team_names = sg[['Team1', 'Team2']].unstack().unique()
        team_check = True
        for name in team_names:
            if name not in teams_id['team'].values:
                print(f'{name} not in teams')
                team_check = False
                break
            id = get_team_id(teams_id, name)
            if id not in teams:
                teams[id] = Team(name, league)
            else:
                teams[id].update_team_name(name)
                if is_proper_league(league):
                    teams[id].update_league(league)
        if not team_check:
            break
        
        for name in team_names:
            id = get_team_id(teams_id, name)
            teams[id].init_rd()

        proceed_rating(teams_id, teams, sg)

    if not team_check:
        break
if team_check:
    rating = get_rating(teams)

Season 1 World Championship rating ...
	28 matches
Champions/2012 Season/Spring rating ...
	44 matches
Champions/2012 Season/Summer rating ...
	46 matches
Season 2/Regional Finals/Europe rating ...
	17 matches
Season 2/Regional Finals/Korea rating ...
	16 matches
Season 2 World Championship rating ...
	31 matches
Champions/2013 Season/Winter rating ...
	112 matches
NA LCS/Season 3/Spring Season rating ...
	112 matches
EU LCS/Season 3/Spring Season rating ...
	112 matches
LPL/2013 Season/Spring Season rating ...
	112 matches
Champions/2013 Season/Spring rating ...
	87 matches
EU LCS/Season 3/Spring Playoffs rating ...
	15 matches
NA LCS/Season 3/Spring Playoffs rating ...
	20 matches
EU LCS/Season 3/Summer Promotion rating ...
	28 matches
NA LCS/Season 3/Summer Promotion rating ...
	24 matches
NA LCS/Season 3/Summer Season rating ...
	113 matches
EU LCS/Season 3/Summer Season rating ...
	118 matches
LPL/2013 Season/Spring Playoffs rating ...
	12 matches
Champions/2013 Season/Summer rati

In [5]:
rating

,Team,League,Win,Loss,WinRate,r,RD,last_game_date
0,DRX,LCK,457,417,0.522883,2205.053640,90.572336,2022-11-06 04:42:00
1,T1,LCK,695,332,0.676728,2188.645785,96.014738,2022-11-06 04:42:00
2,DWG KIA,LCK,330,165,0.666667,2137.013528,131.044175,2022-10-23 00:49:00
3,Gen.G,LCK,503,353,0.587617,2098.365035,112.558777,2022-10-31 00:03:00
4,Samsung White,LTC,94,42,0.691176,2005.948802,128.722083,2014-10-19 09:44:00
...,...,...,...,...,...,...,...,...
584,Jakarta Juggernauts,GPL,0,18,0.000000,410.381826,240.752075,2015-07-10 12:00:00
585,Bencheados,CLS,7,31,0.184211,370.990553,304.945359,2017-03-30 23:23:00
586,Vaevictis eSports,LCL,42,86,0.328125,363.702283,217.284981,2019-08-25 15:08:00
587,Team Infinite,GPL,0,22,0.000000,329.463221,240.149071,2015-01-29 02:00:00


In [6]:
rating.loc[rating['League'] == 'LPL']

,Team,League,Win,Loss,WinRate,r,RD,last_game_date
5,Top Esports,LPL,327,244,0.572680,1908.636296,191.239860,2022-10-16 00:02:00
6,EDward Gaming,LPL,628,337,0.650777,1885.139695,126.572916,2022-10-24 01:33:00
7,JD Gaming,LPL,325,230,0.585586,1864.143061,276.043549,2023-01-14 13:36:00
8,Royal Never Give Up,LPL,567,331,0.631403,1860.122153,155.135549,2022-10-21 23:18:00
23,Invictus Gaming,LPL,536,427,0.556594,1561.102202,259.325909,2023-01-14 10:25:00
27,Bilibili Gaming,LPL,221,225,0.495516,1544.244235,276.043549,2023-01-14 13:36:00
37,LNG Esports,LPL,383,397,0.491026,1483.070439,139.201137,2022-09-04 13:35:00
53,Rare Atom,LPL,291,388,0.428571,1399.689885,228.990837,2023-01-15 09:10:00
54,Victory Five,LPL,140,185,0.430769,1398.782411,212.615082,2022-09-03 12:48:00
62,ThunderTalk Gaming,LPL,108,195,0.356436,1380.644488,63.624314,2022-08-13 10:55:00


In [16]:
rating['last_game_date'] = pd.to_datetime(rating['last_game_date'])

In [17]:
rating.loc[rating['last_game_date'].dt.year == 2023]

,Team,League,Win,Loss,WinRate,r,RD,last_game_date
7,JD Gaming,LPL,325,230,0.585586,1864.143061,276.043549,2023-01-14 13:36:00
23,Invictus Gaming,LPL,536,427,0.556594,1561.102202,259.325909,2023-01-14 10:25:00
27,Bilibili Gaming,LPL,221,225,0.495516,1544.244235,276.043549,2023-01-14 13:36:00
53,Rare Atom,LPL,291,388,0.428571,1399.689885,228.990837,2023-01-15 09:10:00
65,Team WE,LPL,485,456,0.515409,1375.154926,237.712757,2023-01-14 09:07:00
115,FunPlus Phoenix,LPL,302,207,0.593320,1229.585385,237.712757,2023-01-14 09:07:00
172,Anyone's Legend,LPL,167,243,0.407317,1139.149896,259.325909,2023-01-14 10:25:00
194,LGD Gaming,LPL,355,445,0.443750,1105.213027,228.990837,2023-01-15 09:10:00
